In [74]:
import pandas as pd
import joblib
import json
from io import StringIO

import mlflow
import matplotlib.pyplot as plt
from mlflow import MlflowClient
import requests

In [48]:
# mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
mlflow.set_tracking_uri(uri='http://127.0.0.1:8000')
client = MlflowClient()
local_dir = ('C:/Users/Peter/DataspellProjects/pollock/')

In [64]:
data = pd.read_csv('data/original/Covid_data_full.csv')
# we use only part of data which was not used in training and validation
# data = data.iloc[900000:]
data = data.iloc[1048570:]


# let's make this classification task

data.loc[~data['DATE_DIED'].isin(['9999-99-99']), 'DATE_DIED'] = 1 #checks if the  column is not in the list ['9999-99-99'].
data.loc[data['DATE_DIED'] == '9999-99-99', 'DATE_DIED'] = 0


data

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
1048570,2,13,2,1,0,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,0,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,0,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,0,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97
1048574,2,13,2,1,0,97,2,52,97,2,...,2,2,2,2,2,2,2,2,7,97


In [65]:
X = data.drop('DATE_DIED',axis=1)
y = data['DATE_DIED']

##### 1. XGBoost

In [66]:


X_dict = X.reset_index().to_dict(orient='list')

predicted_data_response_xgb = requests.post('http://127.0.0.1:8000/model/predict_xgb',  json=X_dict) # local address
# predicted_data_response = requests.post('https://first-try-6o4cxe37ma-uc.a.run.app/model/predict',  json=X_test_dict) # docker addsess


print(predicted_data_response_xgb)
print(predicted_data_response_xgb.text)

<Response [200]>
"{\"index\":{\"0\":0,\"1\":1,\"2\":2,\"3\":3,\"4\":4},\"0\":{\"0\":0,\"1\":0,\"2\":0,\"3\":0,\"4\":0}}"


In [67]:
predicted_data_response_xgb.json()

'{"index":{"0":0,"1":1,"2":2,"3":3,"4":4},"0":{"0":0,"1":0,"2":0,"3":0,"4":0}}'

In [81]:
predicted_results = pd.read_json(StringIO(predicted_data_response_xgb.json())).set_index('index')
predicted_results

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 2. Catboost

In [80]:
predicted_data_response_catb = requests.post('http://127.0.0.1:8000/model/predict_catb',  json=X_dict) # local address
predicted_results_catb = pd.read_json(StringIO(predicted_data_response_catb.json())).set_index('index')
predicted_results_catb

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 3. LightGBM

In [79]:
predicted_data_response_lgbm = requests.post('http://127.0.0.1:8000/model/predict_lgbm',  json=X_dict) # local address
predicted_results_lgbm = pd.read_json(StringIO(predicted_data_response_lgbm.json())).set_index('index')
predicted_results_lgbm

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 4. KNN

In [78]:
predicted_data_response_knn = requests.post('http://127.0.0.1:8000/model/predict_knn',  json=X_dict) # local address
predicted_results_knn = pd.read_json(StringIO(predicted_data_response_knn.json())).set_index('index')
predicted_results_knn

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 5. Logreg

In [82]:
predicted_data_response_logreg = requests.post('http://127.0.0.1:8000/model/predict_logreg',  json=X_dict) # local address
predicted_results_logreg = pd.read_json(StringIO(predicted_data_response_logreg.json())).set_index('index')
predicted_results_logreg

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 6. Naive Bayes


In [83]:
predicted_data_response_nb = requests.post('http://127.0.0.1:8000/model/predict_nb',  json=X_dict) # local address
predicted_results_nb = pd.read_json(StringIO(predicted_data_response_nb.json())).set_index('index')
predicted_results_nb

,0
index,
0,0
1,0
2,0
3,0
4,0


##### 7. Ensemble

In [84]:
predicted_data_response_ens = requests.post('http://127.0.0.1:8000/model/predict_ens',  json=X_dict) # local address
predicted_results_ens = pd.read_json(StringIO(predicted_data_response_ens.json())).set_index('index')
predicted_results_ens

,0
index,
0,0
1,0
2,0
3,0
4,0
